In [94]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

import os
#os.chdir('../')


In [95]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [96]:
import pickle
import openpyxl
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterSampler, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, recall_score, precision_score
import matplotlib.pyplot as plt


In [97]:
learners = [
    {
        "learner": XGBClassifier,
        "params": {
            "learning_rate": [0.1],
            "colsample_bytree": [0.15, 0.25, 0.5, 0.75, 1],
            "colsample_bylevel": [1],
            "max_depth": [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25],
            "gamma": [0, 0.01, 0.05, 0.1, 0.25, 0.5],
            "subsample": [0.5, 0.75, 1],
            "min_child_weight": [1, 5, 10, 15, 25, 50, 100],
            "base_score": [0.52],
            "n_estimators": [50, 100, 200, 300, 500, 1000] 
            }
    },
    {
        "learner": RandomForestClassifier,
        "params": {
            "max_depth": [3, 4, 5, 10, 25, None],
            "max_features": ["auto", 0.1, 0.25, 0.5, 0.75, 1],
            "min_weight_fraction_leaf": [0, 0.01, 0.001],
            "bootstrap": [True, False],
            "n_jobs": [-1],
            "n_estimators": [50, 100, 200, 300, 500, 1000]             
            }
    },
    {
        "learner": ExtraTreesClassifier,
        "params": {
            "max_depth": [None],
            "max_features": ["auto", 0.1],
            "min_weight_fraction_leaf": [0, 0.01, 0.001],
            "bootstrap": [True, False],
            "n_jobs": [-1],
            "n_estimators": [50, 100, 200, 300, 500, 1000]             
            }
    },
    {
        "learner": LogisticRegression,
        "params": {
            "penalty": ["l2"],
            "C": [0.001, 0.01, 0.1, 1, 10, 100, 100],
            }
    },   
    {
        "learner": LGBMClassifier,
        "params": {
            "boosting_type": ["gbdt", "dart", "goss"],
            "max_depth": [-1, 3, 5, 10, 20],
            "min_child_samples": [20, 30, 40, 50],
            "n_estimators": [50, 100, 200, 300],
            "num_leaves": [20, 31, 50, 80],
            }

    },
]


In [98]:
train = pd.read_csv("./bankMarketing/02 - procesada/bankMarketingTotal.csv",sep=";")

In [99]:
train.head()

,default,housing,loan,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,student_retired,degree,previousProd,marriedAdmin,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,cuartilesAge,decilesAge,bucketsAge
0,0,0,0,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]"
1,0,0,0,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]"
2,0,1,0,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,4,"(30, 40]"
3,0,0,0,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,2,5,"(30, 40]"
4,0,0,1,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]"


In [100]:
train["ID"]=train.index

In [101]:
X, X_validation, y, y_validation = train_test_split(train.drop(columns=["y","bucketsAge"]), train.y, test_size=0.1, random_state=42)

In [102]:
train.head()

,default,housing,loan,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,student_retired,degree,previousProd,marriedAdmin,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,cuartilesAge,decilesAge,bucketsAge,ID
0,0,0,0,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]",0
1,0,0,0,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]",1
2,0,1,0,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,4,"(30, 40]",2
3,0,0,0,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,2,5,"(30, 40]",3
4,0,0,1,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3,9,"(50, 60]",4


In [103]:
skf = StratifiedKFold(n_splits=5, random_state=None)

In [104]:
if not os.path.exists("./clase4/results"):
    os.makedirs("./clase4/results")

if os.path.exists("./clase4/results/results.xlsx"):
    results = pd.read_excel(resultados_path)
    results["parameters"] = [eval(d) for d in results.parameters]
else:
    results = pd.DataFrame()


for x in range(0,10):
    candidate = np.random.choice(learners)
    learnerName = candidate["learner"].__name__
    params = list(ParameterSampler(candidate["params"], 1))[0]

    
    usedParams = results.filter(regex=learnerName , axis = 0)
    if len(usedParams) != 0 and np.any(usedParams.parameters == params):
        continue
    probs = pd.DataFrame()
    z = 0
    for train_index, test_index in skf.split(X,y): 
        X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        learner = candidate["learner"](**params)
        learner.fit(X_train, y_train)        
        aux = pd.DataFrame(X_test.ID, columns = ["ID"])  
        aux["probabilities"] = learner.predict_proba(X_test)[:,1]
        probs = probs.append(aux)
        indexName = "{}_{}".format(learnerName, str(x))
        if not os.path.exists("./clase4/results/{}".format(indexName)):
            os.makedirs("./clase4/results/{}".format(indexName))
        pickle.dump(learner, open("./clase4/results/{}/{}_{}.sav".format(indexName, indexName, str(z)), 'wb')) 
        z+=1
        
        
    probs.to_csv("./clase4/results/{}/probabilities.txt".format(indexName), index = False)
    areaUnderCurve = roc_auc_score(y.values, probs.probabilities.values)
    accuracy = accuracy_score(y.values, probs.probabilities.values > 0.5)
    recall = recall_score(y.values, probs.probabilities.values > 0.5)
    precision = precision_score(y.values, probs.probabilities.values > 0.5)
    row = [str(params), recall, precision, areaUnderCurve, accuracy]
    results = results.append(pd.DataFrame([row], columns=["parameters", "recall", "precision", "areaUnderCurve", "accuracy"],index=[indexName]))
    

results.to_excel("./clase4/results/results.xlsx")




In [105]:
models = os.listdir("./clase4/results/")
X_validationNew = X_validation[["ID"]]
for model in models:
    if model == "results.xlsx":
        continue
    model1 = pickle.load(open("./clase4/results/{}/{}_0.sav".format(model, model), 'rb'))
    model2 = pickle.load(open("./clase4/results/{}/{}_1.sav".format(model, model), 'rb'))
    model3 = pickle.load(open("./clase4/results/{}/{}_2.sav".format(model, model), 'rb'))
    model4 = pickle.load(open("./clase4/results/{}/{}_3.sav".format(model, model), 'rb'))
    model5 = pickle.load(open("./clase4/results/{}/{}_4.sav".format(model, model), 'rb'))
    aux = X_validation[["ID"]]
    aux["score_0"] = model1.predict_proba(X_validation)[:,1]
    aux["score_1"] = model2.predict_proba(X_validation)[:,1]
    aux["score_2"] = model3.predict_proba(X_validation)[:,1]
    aux["score_3"] = model4.predict_proba(X_validation)[:,1]
    aux["score_4"] = model5.predict_proba(X_validation)[:,1]
    aux["probabilities_{}".format(model)] = (aux.score_1+aux.score_2+aux.score_3+aux.score_4+aux.score_0) / 5
    X_validationNew = pd.merge(X_validationNew, aux[["ID", "probabilities_{}".format(model)]], 
                               how="inner",on="ID")

    

In [108]:
X_validationNew


,ID,probabilities_ExtraTreesClassifier_4,probabilities_ExtraTreesClassifier_7,probabilities_LGBMClassifier_1,probabilities_LGBMClassifier_6,probabilities_LogisticRegression_2,probabilities_LogisticRegression_8,probabilities_RandomForestClassifier_0,probabilities_RandomForestClassifier_5,probabilities_XGBClassifier_3,probabilities_XGBClassifier_9
0,32884,0.0600,0.068,0.041414,3.573060e-02,0.124342,0.133326,0.111663,0.076891,0.048450,0.089985
1,3169,0.0028,0.000,0.002488,5.155142e-05,0.013158,0.012542,0.074534,0.029816,0.010800,0.031510
2,32206,0.0016,0.004,0.001460,2.053997e-07,0.047979,0.047225,0.100514,0.067683,0.012709,0.041900
3,9403,0.0236,0.044,0.006241,1.067166e-02,0.022973,0.022432,0.075142,0.032892,0.017139,0.046118
4,14020,0.0000,0.000,0.003093,2.754994e-05,0.019695,0.018834,0.091093,0.041704,0.013139,0.049740
...,...,...,...,...,...,...,...,...,...,...,...
4114,36295,0.5804,0.584,0.584245,6.095817e-01,0.368110,0.377633,0.275279,0.464299,0.676053,0.440663
4115,1553,0.0000,0.000,0.001530,3.490939e-07,0.005087,0.004918,0.071538,0.028730,0.008384,0.021763
4116,24928,0.0760,0.112,0.246022,8.341755e-02,0.207843,0.213970,0.118956,0.175171,0.183312,0.147827
4117,32493,0.0012,0.000,0.015072,1.484448e-03,0.093336,0.092899,0.106620,0.072420,0.020518,0.078289


In [106]:
models = os.listdir("./clase4/results/")
probabilities = pd.read_csv("./clase4/results/{}/probabilities.txt".format(models[0]))
probabilities = probabilities.rename(columns={"probabilities": "probabilities_{}".format(models[0])})

for model in models[1:]:
    if model == "results.xlsx":
        continue
    aux = pd.read_csv("./clase4/results/{}/probabilities.txt".format(model))
    aux = aux.rename(columns={"probabilities": "probabilities_{}".format(model)})
    probabilities = pd.merge(probabilities, aux, how="inner", on="ID")
    

In [107]:
learner = XGBClassifier()
learner.fit(probabilities.drop(columns=["ID"]), y)





XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)